In [1]:
using Oscar;
using Combinatorics;
pm = Polymake;

In [2]:
include("computing_Iw_Omega.jl")
include("gfan_functions.jl")

ideal_to_gfan (generic function with 3 methods)

## Schubert varieties

Let $1\leq r \leq n$ be integers and let $B = (b_1, \ldots, b_r)$ be a $r$-element list of distinct elements of $\{1,\ldots,n\}$ in increasing order. The Schubert variety is

$$
X_{B} = \{L\in \mathrm{Gr}(r,n) : p_{A}(L) = 0 \text{ for } A \nleq B\}. 
$$

If $I_{r,n}$ denotes the Plücker ideal of $\mathrm{Gr}(r,n)$, the ideal of $X_{B}$ is given by

$$
I(X_{B}) = I_{r,n} + \langle p_{A} :  A \nleq B \rangle 
$$

In the function `schubert_variety` below, we construct the Plücker ideal of $X_B$. There is an option `elim` which, if marked `true`, will eliminate the variables $p_{A}$ for  $A \nleq B$. Note that this coincides with applying the map $p_{A}\mapsto 0$ to the ideal $I_{r,n}$. In any case, this produces a realization of  $X_B$ in a smaller coordinate subspace of the ambient Plücker projective space. 

In [3]:
function partial_order_lists(A, B)
    if length(A) != length(B)
        return false
    elseif all([A[i] <= B[i] for i in 1:length(A)])
        return true
    else
        return false
    end
end

function lex_index(r, n)
    A = sort!(subsets(n,r))
    return Dict(A[i] => i for i in 1:length(A))
end

function remove_vars(I, vars_to_keep_index)
    R = base_ring(I)
    x = gens(R)
    m = length(vars_to_keep_index)
    vars_to_keep = x[vars_to_keep_index]
    S, y = polynomial_ring(coefficient_ring(R), ["y$(i)" for i in 1:m])
    phi = hom(S, R, vars_to_keep)
    return preimage(phi, I)
end

function schubert_variety(r, n, B, elim=false)
    I = [A for A in subsets(n, r) if !partial_order_lists(A, B)]
    sort!(setdiff( subsets(n, r), I))
    ncr = binomial(n,r) 
    R, x = polynomial_ring(QQ, ncr)
    Gr = grassmann_pluecker_ideal(R, r, n)
    S = base_ring(Gr)
    R = forget_grading(S)
    phi = hom(R, S, gens(S))  
    Gr = preimage(phi, Gr)
    x = gens(R)
    ind = lex_index(r, n)
    if length(I) != 0
        J = [x[ind[A]] for A in I]
    else
        J =[]
    end
    Jc = sort!([ind[A] for A in keys(ind) if !(A in I) ])
    K = Gr + ideal(R, J)
    if elim
        if length(J) != 0
            Kelim = eliminate(K, J)
        else
            Kelim = K
        end
        return remove_vars(Kelim, Jc)
    else
        return K
    end
end

schubert_variety (generic function with 2 methods)

## Point configurations for Schubert varieties

The function `schubert_lineality_space` computes the point configuration of the schubert variety $X_B$. The point configuration of the Grassmannian consists of the vertices of the hypersimplex, and these vertices are labeled by the $r$ element subsets of $\{1,\ldots,n\}$, so the point configuiration of $X_B$ consists of those vertices corresponding to the subsets $A$ satisfying $A\leq B$. 

In [4]:
function grassmannian_lineality_space(r, n)
    L = zeros(Int, n, binomial(n,r))
    j=1
    for A in sort!(subsets(n, r))
        L[A, [j]] = ones(Int, r, 1)
        j+=1
    end
    return L
end

function schubert_lineality_space(r, n, B)
    I = [A for A in sort!(subsets(n, r)) if partial_order_lists(A, B)]
    L = zeros(Int, length(I), n)
    j=1
    for A in I
        L[[j], A] = ones(Int, 1, r)
        j+=1
    end
    return L
end

schubert_lineality_space (generic function with 1 method)

In [5]:
function stepwise_saturation(I, gs)
    for a in gs
        I = saturation(I, ideal(a))
    end
    return I
end

stepwise_saturation (generic function with 1 method)

In [6]:
function ray_cone_to_reps(rs, cs)
    n_cones, n_rays = size(cs)
    d = length(rs[1])
    reps = []
    for i in 1:n_cones
        w = [rs[j] for j in 1:n_rays if cs[i,j]]
        if length(w) == 0
            w = zeros(Int, d)
        else
            w = sum(w)
        end
        push!(reps, w)
    end
    return reps
end

function Omega_fan(I, Delta, Sec, sat = false)
    if typeof(Sec) == PolyhedralFan{QQFieldElem}
        rays_Delta = rays_modulo_lineality(Sec)[1];
        cones_Delta = cones(Sec);
    else
        rays_Delta, cones_Delta = Sec
    end

    n_cones_Delta, n_rays_Delta = size(cones_Delta)
    reps = ray_cone_to_reps(rays_Delta, cones_Delta)

    tests = [true]
    for w in reps[2:length(reps)]
        w = -Vector{Int64}(pm.common.primitive(w))
        init_w_I = initial(I, nu_t, w)
        I_w = ideal_w(I,  w, Delta)
        push!(tests, init_w_I == I_w)
        #@show tests
    end
    inside_Omega = [i for i in 1:length(reps) if tests[i]];
    return [rays_Delta, cones_Delta[inside_Omega, :]]  
end

Omega_fan (generic function with 2 methods)

In [8]:
r=3; n=7; B=[2,4,7]
Bs = join([string(a) for a in B])
just_file = "schubert_r$(r)_n$(n)_B$(Bs).trop"
file_name = joinpath("schubert", "tropicalizations_nocones", just_file)
rs, cs = parse_tropical_file(file_name, true)
point_conf = schubert_lineality_space(r, n, B)
IS = schubert_variety(r, n, B, true)
ray_cone_to_reps(rs, cs)
O = Omega_fan(IS, point_conf, [rs, cs], false)
@show O
#@show cs;

O = AbstractArray[[[-72, 30, 14, 14, 14, 21, 5, 5, 5, -12, -12, -12, 21, 5, 5, 5, -12, -12, -12], [6, 6, -4, -4, -4, -6, 1, 1, 1, 1, 1, 1, -6, 1, 1, 1, 1, 1, 1], [30, -72, 14, 14, 14, 21, -12, -12, -12, 5, 5, 5, 21, -12, -12, -12, 5, 5, 5]], 4×3 IncidenceMatrix
[]
[1]
[2]
[3]
]


2-element Vector{AbstractArray}:
 [[-72, 30, 14, 14, 14, 21, 5, 5, 5, -12, -12, -12, 21, 5, 5, 5, -12, -12, -12], [6, 6, -4, -4, -4, -6, 1, 1, 1, 1, 1, 1, -6, 1, 1, 1, 1, 1, 1], [30, -72, 14, 14, 14, 21, -12, -12, -12, 5, 5, 5, 21, -12, -12, -12, 5, 5, 5]]
 4×3 IncidenceMatrix
[]
[1]
[2]
[3]


In [33]:
just_file = "Omega_tropical_no_saturation_cones_r$(r)_n$(n)_B$(Bs).mrdi"
file_name = joinpath("schubert", "omega_in_tropical_fan", just_file)
save(file_name, O[2]);

In [14]:
r=3; n=7; B=[3,5,7]
@show string(sort!([A for A in subsets(n, r) if !partial_order_lists(A, B)]))
string(sort!([A for A in subsets(n, r) if partial_order_lists(A, B)]))

string(sort!([A for A = subsets(n, r) if !(partial_order_lists(A, B))])) = "[[1, 6, 7], [2, 6, 7], [3, 6, 7], [4, 5, 6], [4, 5, 7], [4, 6, 7], [5, 6, 7]]"


"[[1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 2, 6], [1, 2, 7], [1, 3, 4], [1, 3, 5], [1, 3, 6], [1, 3, 7], [1, 4, 5], [1, 4, 6], [1, 4, 7], [1, 5, 6], [1, 5, 7], [2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 3, 7], [2, 4, 5], [2, 4, 6], [2, 4, 7], [2, 5, 6], [2, 5, 7], [3, 4, 5], [3, 4, 6], [3, 4, 7], [3, 5, 6], [3, 5, 7]]"

In [39]:
r=3; n=7; B=[3,6,7]
Bs = join([string(a) for a in B])
just_file = "Omega_tropical_no_saturation_cones_r$(r)_n$(n)_B$(Bs).mrdi"
file_name = joinpath("schubert", "omega_in_tropical_fan", just_file)
cs = load(file_name)
#pm.to_zero_based_indexing(cs)

5×56 IncidenceMatrix
[]
[5]
[56]
[54, 56]
[51, 54, 56]


In [ ]:
r = 3; n = 9;# B = [4,6];
for B in sort!(subsets(n,r))
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    if iszero(S) || last(B) != n
        continue
    end
    just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    file_name = joinpath("schubert", "initial_files", just_file)
    println(file_name)
    ideal_to_gfan(S, true, file_name)
end

schubert/initial_files/schubert_r2_n9_B29.dat
schubert/initial_files/schubert_r2_n9_B39.dat
schubert/initial_files/schubert_r2_n9_B49.dat
schubert/initial_files/schubert_r2_n9_B59.dat
schubert/initial_files/schubert_r2_n9_B69.dat
schubert/initial_files/schubert_r2_n9_B79.dat
schubert/initial_files/schubert_r2_n9_B89.dat


In [10]:
r = 2; n = 7; #B = [3,4];
for B in sort!(subsets(n,r))
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    if iszero(S)
        continue
    end
    in_just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    in_file_name = joinpath("schubert", "initial_files", in_just_file)
    out_just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    out_file_name = joinpath("schubert", "start_cones", out_just_file)
    @show B
    open(out_file_name, "w") do out
        open(in_file_name, "r") do inp
            run(pipeline(`gfan_tropicalstartingcone`, stdin=inp, stdout=out))
        end
    end
end

I = [A for A = subsets(n, r) if !(partial_order_lists(A, B))] = [[1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 5], [2, 5], [3, 5], [4, 5], [1, 4], [2, 4], [3, 4], [1, 3], [2, 3]]
sort!(setdiff(subsets(n, r), I)) = [[1, 2]]
I = [A for A = subsets(n, r) if !(partial_order_lists(A, B))] = [[1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 5], [2, 5], [3, 5], [4, 5], [1, 4], [2, 4], [3, 4], [2, 3]]
sort!(setdiff(subsets(n, r), I)) = [[1, 2], [1, 3]]
I = [A for A = subsets(n, r) if !(partial_order_lists(A, B))] = [[1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 5], [2, 5], [3, 5], [4, 5], [2, 4], [3, 4], [2, 3]]
sort!(setdiff(subsets(n, r), I)) = [[1, 2], [1, 3], [1, 4]]
I = [A for A = subsets(n, r) if !(partial_order_lists(A, B))] = [[1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [2, 5], [3, 5], [4, 5], [2, 4], [3, 4], 

In [121]:
r = 3; n = 6; #B = [3,4];
for B in subsets(n,r)
    Bs = join([string(a) for a in B])
    point_conf = schubert_lineality_space(r, n, B)
    point_conf_str = point_configuration_to_string(point_conf)
    just_file = "point_config_r$(r)_n$(n)_B$(Bs).dat"
    file_name = joinpath("schubert", "point_configurations", just_file)
    point_conf_str = point_configuration_to_string(point_conf)
    open(file_name, "w") do out_f
        write(out_f, point_conf_str)
    end
end



In [ ]:
r = 2; n = 6; #B = [3,4];
for B in subsets(n,r)
    Bs = join([string(a) for a in B])
    in_just_file = "point_config_r$(r)_n$(n)_B$(Bs).dat"
    in_file_name = joinpath("schubert", "point_configurations", in_just_file)
    out_just_file = "secondary_r$(r)_n$(n)_B$(Bs).dat"
    out_file_name = joinpath("schubert", "secondary_fans", out_just_file)
    open(out_file_name, "w") do out
        open(in_file_name, "r") do inp
            #run(pipeline(`gfan_secondaryfan`, stdin=inp, stdout=out))
        end
    end
end
    

I = [A for A = subsets(n, r) if !(partial_order_lists(A, B))] = Vector{Int64}[]
sort!(setdiff(subsets(n, r), I)) = [[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [2, 3], [2, 4], [2, 5], [2, 6], [3, 4], [3, 5], [3, 6], [4, 5], [4, 6], [5, 6]]


MethodError: MethodError: no method matching zero(::Type{Any})
This error has been manually thrown, explicitly, so the method may exist but be intentionally marked as unimplemented.

Closest candidates are:
  zero(::Type{Union{Missing, T}}) where T
   @ Base missing.jl:105
  zero(!Matched::Type{Union{}}, Any...)
   @ Base number.jl:310
  zero(!Matched::Type{Dates.Date})
   @ Dates /Applications/Julia-1.11.app/Contents/Resources/julia/share/julia/stdlib/v1.11/Dates/src/types.jl:459
  ...


In [ ]:
Delta_d = point_configuration(I_d)
Omega_d = Omega_fan(I_d, Delta_d, Sec_d)


In [76]:
S = schubert_variety(2, 6, [3,6], true)
#ideal_to_gfan(S)
groebner_basis(S)

Gröbner basis with elements
  1: y9*y11 - y8*y12
  2: y5*y11 - y4*y12
  3: y9*y10 - y7*y12
  4: y8*y10 - y7*y11
  5: y5*y10 - y3*y12
  6: y4*y10 - y3*y11
  7: y5*y8 - y4*y9
  8: y5*y7 - y3*y9
  9: y4*y7 - y3*y8
  10: y5*y6 - y2*y9 + y1*y12
  11: y4*y6 - y2*y8 + y1*y11
  12: y3*y6 - y2*y7 + y1*y10
with respect to the ordering
  degrevlex([y1, y2, y3, y4, y5, y6, y7, y8, y9, y10, y11, y12])

In [4]:
Q = [A for A in subsets(6, 2) if partial_order_lists(A, [3,6])]
ind = lex_index(2,6)
J = sort!([ind[A] for A in Q])
#length(J)
S = schubert_variety(2, 6, [3, 6], true)
base_ring(S)

Multivariate polynomial ring in 12 variables y01, y02, y03, y04, ..., y12
  over rational field

In [404]:
for a in subsets(6,3)
    S = schubert_variety(3, 6, a, true)
    if !is_zero(S)
        file_name = "schubert/initial_files/schubert_36_" * join([string(z) for z in a], "") * ".dat" 
        println(file_name)
        ideal_to_gfan(S, true, file_name)
        cone_name = "schubert/start_cones/schubert_36_" * join([string(z) for z in a], "") * ".cone" 
        trop_name = "schubert/tropicalizations/schubert_36_" * join([string(z) for z in a], "") * ".trop" 
        #trop_start = `gfan_tropicalstartingcone < $(file_name) > $(cone_name)`
        #trop_trav = `gfan_tropicalstartingcone < $(cone_name) > $(trop_name)`
        
        run(pipeline(`gfan_tropicalstartingcone`,
              stdin=open(file_name, "r"),
              stdout=open(cone_name, "w")))

        run(pipeline(`gfan_tropicaltraverse`,
              stdin=open(cone_name, "r"),
              stdout=open(trop_name, "w")))
    end
end

schubert/initial_files/schubert_26_156.dat
schubert/initial_files/schubert_26_256.dat
schubert/initial_files/schubert_26_356.dat
schubert/initial_files/schubert_26_456.dat
schubert/initial_files/schubert_26_146.dat
schubert/initial_files/schubert_26_246.dat
schubert/initial_files/schubert_26_346.dat
schubert/initial_files/schubert_26_136.dat
schubert/initial_files/schubert_26_236.dat
schubert/initial_files/schubert_26_145.dat
schubert/initial_files/schubert_26_245.dat
schubert/initial_files/schubert_26_345.dat
schubert/initial_files/schubert_26_135.dat
schubert/initial_files/schubert_26_235.dat
